In [2]:
%pip install -q langchain-chroma 
%pip install -q langchain-community
%pip install -q python-dotenv 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -q transformers 
%pip install -q bitsandbytes 
%pip install -q accelerate 
%pip install -q langchain 
%pip install -q langchainhub 
%pip install -q langchain-chroma 
%pip install -q langchain-community 
%pip install -q langchain_huggingface 
%pip install -q python-dotenv 
%pip install -q pypdf 
%pip install -q numpy 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'hroma'
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'ommunity'
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Usage:   
  /opt/anaconda3/envs/python3/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /opt/anaconda3/envs/python3/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /opt/anaconda3/envs/python3/bin/python -m pip ins

In [1]:
import torch

# from transformers import BitsAndBytesConfig
# from transformers import AutoTokenizer , AutoModelForCausalLM, pipeline
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_huggingface.llms import HuggingFacePipeline

# from langchain.memory import ConversationBufferMemory
# from langchain_community.chat_message_histories import ChatMessageHistory
# from langchain_community.document_loaders import PyPDFLoader ,TextLoader
# from langchain.chains import ConversationalRetrievalChain
# from langchain_chroma import Chroma
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
# from langchain import hub

ImportError: dlopen(/opt/anaconda3/envs/python3/lib/python3.13/site-packages/numpy/linalg/_umath_linalg.cpython-313-darwin.so, 0x0002): symbol not found in flat namespace '_ccopy$NEWLAPACK_'